<a href="https://colab.research.google.com/github/ditarnnt/Telecom-Predic-Churn/blob/main/PP_Orange.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##IMPORT LIBRARY

In [ ]:
from google.colab import drive
import pandas as pd #pandas
import numpy as np #numpy
from sklearn.preprocessing import LabelEncoder, OneHotEncoder #encoding data kategorikal
from sklearn.preprocessing import StandardScaler #standardization
from sklearn.model_selection import train_test_split #train test split

##IMPORT DATASET

In [ ]:
drive.mount('/content/drive')

orange_big = pd.read_csv('/content/drive/My Drive/Colab Notebooks/churn-bigml-80.csv',keep_default_na=False,na_values=[""])
orange_small = pd.read_csv('/content/drive/My Drive/Colab Notebooks/churn-bigml-20.csv',keep_default_na=False,na_values=[""])

orange = orange_big.append(orange_small)
orange

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,WI,114,415,No,Yes,26,137.1,88,23.31,155.7,125,13.23,247.6,94,11.14,11.5,7,3.11,2,False
663,AL,106,408,No,Yes,29,83.6,131,14.21,203.9,131,17.33,229.5,73,10.33,8.1,3,2.19,1,False
664,VT,60,415,No,No,0,193.9,118,32.96,85.0,110,7.23,210.1,134,9.45,13.2,8,3.56,3,False
665,WV,159,415,No,No,0,169.8,114,28.87,197.7,105,16.80,193.7,82,8.72,11.6,4,3.13,1,False


##PREPROCESSING

In [ ]:
categorical_columns_orange = ['State','Area code','International plan','Voice mail plan','Churn'] #fitur-fitur yang kategorikal

drop_cat_cols_orange = [] #placeholder untuk categorical yang bakalan didrop
enc_cat_cols_orange = [] #placeholder untuk categorical yang bakalan lanjut di-encode

for i in range(len(categorical_columns_orange)):
  cat_col_i_orange = categorical_columns_orange[i]
  value_count_orange = orange[cat_col_i_orange].value_counts()
  #print(cat_col_i,":")
  #print(value_count)
  if (len(value_count_orange)>3):         # Selain jenis data yes/no
    drop_cat_cols_orange.append(cat_col_i_orange)

print('Categorical columns to drop:')
print(drop_cat_cols_orange)

enc_cat_cols_orange = list(set(categorical_columns_orange) - set(drop_cat_cols_orange))

print('Categorical columns to encode:')
print(enc_cat_cols_orange)



Categorical columns to drop:
['State']
Categorical columns to encode:
['Churn', 'Voice mail plan', 'International plan', 'Area code']


In [ ]:
#melakukan cleaning dan filtering fitur-fitur tidak berguna
orange.dropna(axis='index',inplace=True)
for dropped_col_orange in drop_cat_cols_orange:
  orange.drop(dropped_col_orange,axis='columns',inplace=True)
orange.drop('Area code',axis='columns',inplace=True) #drop Area code, kasus khusus


orange

,Account length,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,128,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,107,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,137,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,84,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,75,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,114,No,Yes,26,137.1,88,23.31,155.7,125,13.23,247.6,94,11.14,11.5,7,3.11,2,False
663,106,No,Yes,29,83.6,131,14.21,203.9,131,17.33,229.5,73,10.33,8.1,3,2.19,1,False
664,60,No,No,0,193.9,118,32.96,85.0,110,7.23,210.1,134,9.45,13.2,8,3.56,3,False
665,159,No,No,0,169.8,114,28.87,197.7,105,16.80,193.7,82,8.72,11.6,4,3.13,1,False


In [ ]:
#menggeser fitur-fitur kategorikal ke sebelah kanan dataset agar mudah

label_orange = ['Churn'] #label
num_cols_orange = orange.columns.to_list()

def diff(first, second):
  second = set(second)
  return [item for item in first if item not in second]

num_cols_orange = diff(num_cols_orange,label_orange)
num_cols_orange = diff(num_cols_orange,enc_cat_cols_orange) #num_cols adalah list fitur2 numerik
enc_cat_cols_orange = diff(enc_cat_cols_orange, label_orange)
enc_cat_cols_orange = list(set(enc_cat_cols_orange)-set(['Area code']))

cols_orange = num_cols_orange + enc_cat_cols_orange + label_orange #list kolom-kolom yang ordered, termasuk di dalamnya label
print(cols_orange)

orange = orange[cols_orange] #dataframe yang udah diorder

features_orange = diff(cols_orange,label_orange) #fitur-fitur

orange = orange.loc[:,~orange.columns.duplicated()]
orange

['Account length', 'Number vmail messages', 'Total day minutes', 'Total day calls', 'Total day charge', 'Total eve minutes', 'Total eve calls', 'Total eve charge', 'Total night minutes', 'Total night calls', 'Total night charge', 'Total intl minutes', 'Total intl calls', 'Total intl charge', 'Customer service calls', 'Voice mail plan', 'International plan', 'Churn']


,Account length,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Voice mail plan,International plan,Churn
0,128,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,Yes,No,False
1,107,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,Yes,No,False
2,137,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,No,No,False
3,84,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,No,Yes,False
4,75,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,No,Yes,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,114,26,137.1,88,23.31,155.7,125,13.23,247.6,94,11.14,11.5,7,3.11,2,Yes,No,False
663,106,29,83.6,131,14.21,203.9,131,17.33,229.5,73,10.33,8.1,3,2.19,1,Yes,No,False
664,60,0,193.9,118,32.96,85.0,110,7.23,210.1,134,9.45,13.2,8,3.56,3,No,No,False
665,159,0,169.8,114,28.87,197.7,105,16.80,193.7,82,8.72,11.6,4,3.13,1,No,No,False


In [ ]:
print(num_cols_orange,'\n', enc_cat_cols_orange,'\n', label_orange)

['Account length', 'Number vmail messages', 'Total day minutes', 'Total day calls', 'Total day charge', 'Total eve minutes', 'Total eve calls', 'Total eve charge', 'Total night minutes', 'Total night calls', 'Total night charge', 'Total intl minutes', 'Total intl calls', 'Total intl charge', 'Customer service calls'] 
 ['Voice mail plan', 'International plan'] 
 ['Churn']


In [ ]:
#melakukan encoding feature-feature kategorikal
for i in range(-1*len(enc_cat_cols_orange)-1,0):
  labelencoder_orange = LabelEncoder() #object label encoder
  print(i)
  orange.iloc[:,i] = labelencoder_orange.fit_transform(orange.iloc[:,i]) #encode label
 
orange

-3
-2
-1


,Account length,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Voice mail plan,International plan,Churn
0,128,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,1,0,0
1,107,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,1,0,0
2,137,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0,0,0
3,84,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,0,1,0
4,75,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,114,26,137.1,88,23.31,155.7,125,13.23,247.6,94,11.14,11.5,7,3.11,2,1,0,0
663,106,29,83.6,131,14.21,203.9,131,17.33,229.5,73,10.33,8.1,3,2.19,1,1,0,0
664,60,0,193.9,118,32.96,85.0,110,7.23,210.1,134,9.45,13.2,8,3.56,3,0,0,0
665,159,0,169.8,114,28.87,197.7,105,16.80,193.7,82,8.72,11.6,4,3.13,1,0,0,0


In [ ]:
#Split jadi feature dan label

X_orange = np.array(orange.drop('Churn',axis='columns',inplace=False).values)
Y_orange = np.array(orange.loc[:,'Churn'].values)

print(X_orange)
print(Y_orange)

[[128.   25.  265.1 ...   1.    1.    0. ]
 [107.   26.  161.6 ...   1.    1.    0. ]
 [137.    0.  243.4 ...   0.    0.    0. ]
 ...
 [ 60.    0.  193.9 ...   3.    0.    0. ]
 [159.    0.  169.8 ...   1.    0.    0. ]
 [184.    0.  213.8 ...   2.    0.    1. ]]
[0 0 0 ... 0 0 0]


In [ ]:
#INI KALAU OUTPUT CHURN HARUS 'Y' ATAU 'N' BUKAN '1' DAN '0'
#kalau engga dicomment aja
'''
def mapchurn(x):
  if (x==1):
    return 'Y'
  else:
    return 'N'

Y_temp_orange = np.empty(Y_orange.shape,dtype=str)
for i in range(len(Y_orange)):
  Y_temp_orange[i] = mapchurn(Y_orange[i])

print(Y_orange)
print(Y_temp_orange)
Y_orange = Y_temp_orange #Label churn sekarang berupa 'Y' atau 'N'
Y_orange
'''

"\ndef mapchurn(x):\n  if (x==1):\n    return 'Y'\n  else:\n    return 'N'\n\nY_temp_orange = np.empty(Y_orange.shape,dtype=str)\nfor i in range(len(Y_orange)):\n  Y_temp_orange[i] = mapchurn(Y_orange[i])\n\nprint(Y_orange)\nprint(Y_temp_orange)\nY_orange = Y_temp_orange #Label churn sekarang berupa 'Y' atau 'N'\nY_orange\n"

In [ ]:
#PREPROCESSING SELESAI
#Penyesuaian dimensi:

X_orange = X_orange.T
Y_orange = Y_orange.reshape(1,-1)

#DESKRIPSI:
#X adalah matriks fitur dataset dengan size FxM
#Baris F adalah fitur-fitur dataset sejumlah 89 fitur
#Kolom M adalah sample-sample dari dataset

#Y adalah array label setiap sample dengan size 1xM
#M adalah sample-sample pada dataset

print(X_orange)
print(X_orange.shape)
print(Y_orange)
print(Y_orange.shape)

[[128.  107.  137.  ...  60.  159.  184. ]
 [ 25.   26.    0.  ...   0.    0.    0. ]
 [265.1 161.6 243.4 ... 193.9 169.8 213.8]
 ...
 [  1.    1.    0.  ...   3.    1.    2. ]
 [  1.    1.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    1. ]]
(17, 3333)
[[0 0 0 ... 0 0 0]]
(1, 3333)


In [ ]:
orange_preprocessed = pd.DataFrame(np.append(X_orange.T,Y_orange.T,axis=1),columns=cols_orange)
print(orange_preprocessed) #dataset yang sudah dipreprocess


      Account length  Number vmail messages  ...  International plan  Churn
0              128.0                   25.0  ...                 0.0    0.0
1              107.0                   26.0  ...                 0.0    0.0
2              137.0                    0.0  ...                 0.0    0.0
3               84.0                    0.0  ...                 1.0    0.0
4               75.0                    0.0  ...                 1.0    0.0
...              ...                    ...  ...                 ...    ...
3328           114.0                   26.0  ...                 0.0    0.0
3329           106.0                   29.0  ...                 0.0    0.0
3330            60.0                    0.0  ...                 0.0    0.0
3331           159.0                    0.0  ...                 0.0    0.0
3332           184.0                    0.0  ...                 1.0    0.0

[3333 rows x 18 columns]


In [ ]:
orange_preprocessed = pd.DataFrame(np.append(X_orange.T,Y_orange.T,axis=1),columns=cols_orange)
print(orange_preprocessed) #dataset yang sudah dipreprocess

orange_preprocessed.to_csv('/content/drive/My Drive/Colab Notebooks/orange_preprocessed.csv',index=False) #export ke folder TubesAI_folder di drive

      Account length  Number vmail messages  ...  International plan  Churn
0              128.0                   25.0  ...                 0.0    0.0
1              107.0                   26.0  ...                 0.0    0.0
2              137.0                    0.0  ...                 0.0    0.0
3               84.0                    0.0  ...                 1.0    0.0
4               75.0                    0.0  ...                 1.0    0.0
...              ...                    ...  ...                 ...    ...
3328           114.0                   26.0  ...                 0.0    0.0
3329           106.0                   29.0  ...                 0.0    0.0
3330            60.0                    0.0  ...                 0.0    0.0
3331           159.0                    0.0  ...                 0.0    0.0
3332           184.0                    0.0  ...                 1.0    0.0

[3333 rows x 18 columns]


##HASIL FEATURE SELECTION DI WEKA

In [ ]:
'''
=== Attribute Selection on all input data ===

Search Method:
	Attribute ranking.

Attribute Evaluator (supervised, Class (nominal): 18 Churn):
	Information Gain Ranking Filter

Ranked attributes:
 0.0774    3 Total day minutes
 0.0774    5 Total day charge
 0.05009  15 Customer service calls
 0.03688  17 International plan
 0.00822  16 Voice mail plan
 0.00822   2 Number vmail messages
 0.00729  13 Total intl calls
 0.00674  12 Total intl minutes
 0.00674  14 Total intl charge
 0.00542   8 Total eve charge
 0.00542   6 Total eve minutes
 0         4 Total day calls
 0         9 Total night minutes
 0         7 Total eve calls
 0        10 Total night calls
 0        11 Total night charge
 0         1 Account length

Selected attributes: 3,5,15,17,16,2,13,12,14,8,6,4,9,7,10,11,1 : 17
'''

'\n=== Attribute Selection on all input data ===\n\nSearch Method:\n\tAttribute ranking.\n\nAttribute Evaluator (supervised, Class (nominal): 18 Churn):\n\tInformation Gain Ranking Filter\n\nRanked attributes:\n 0.0774    3 Total day minutes\n 0.0774    5 Total day charge\n 0.05009  15 Customer service calls\n 0.03688  17 International plan\n 0.00822  16 Voice mail plan\n 0.00822   2 Number vmail messages\n 0.00729  13 Total intl calls\n 0.00674  12 Total intl minutes\n 0.00674  14 Total intl charge\n 0.00542   8 Total eve charge\n 0.00542   6 Total eve minutes\n 0         4 Total day calls\n 0         9 Total night minutes\n 0         7 Total eve calls\n 0        10 Total night calls\n 0        11 Total night charge\n 0         1 Account length\n\nSelected attributes: 3,5,15,17,16,2,13,12,14,8,6,4,9,7,10,11,1 : 17\n'